In [ ]:
# !pip-compile --strip-extras requirements.in
# %pip install -r requirements.txt

In [1]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-tiny")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-tiny")

2025-06-07 20:52:44.781168: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import DataLoader
from torchaudio.datasets import LIBRISPEECH
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Cek device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset LibriSpeech (dev-clean)
train_dataset = LIBRISPEECH("./datasets", url="dev-clean", download=False) # Jika blm punya dataset, bisa set True
test_dataset = LIBRISPEECH("./datasets", url="test-clean", download=False)

In [5]:
from tqdm import tqdm

model.to(device)
model.train()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(1):  # num_train_epochs
    for i, (waveform, sample_rate, transcript, _, _, _) in enumerate(tqdm(train_dataset)):
        # Preprocessing input
        inputs = processor(waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt")
        input_features = inputs.input_features.to(device)
        labels = processor.tokenizer(transcript, return_tensors="pt").input_ids.to(device)

        # Forward pass
        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss: {loss.item()}")

  1%|          | 1/124 [00:01<04:03,  1.98s/it]

Epoch 0, Step 0, Loss: 5.141490936279297


  9%|▉         | 11/124 [00:12<01:56,  1.03s/it]

Epoch 0, Step 10, Loss: 1.4457511901855469


 17%|█▋        | 21/124 [00:24<02:03,  1.20s/it]

...

 90%|████████▉ | 111/124 [02:11<00:14,  1.13s/it]

Epoch 0, Step 110, Loss: 1.242854356765747


 98%|█████████▊| 121/124 [02:21<00:03,  1.08s/it]

Epoch 0, Step 120, Loss: 1.0097301006317139


100%|██████████| 124/124 [02:24<00:00,  1.17s/it]


In [6]:
# Preprocess the waveform for the model
inputs = processor(waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt")
input_features = inputs.input_features.to(device)

# Generate prediction
model.eval()
with torch.no_grad():
    predicted_ids = model.generate(input_features)

# Decode the prediction
predicted_transcript = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print("Predicted transcript:", predicted_transcript)
print("Reference transcript:", transcript)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Predicted transcript:  AND WHAT ALORMENTS OR WHAT VANTUGE IS UPON THE FOREHEAD OF THE OTHER SHOWED THAT VAL SHOULD'S TURN THY FOOTSTEPPES UNTO THEM
Reference transcript: AND WHAT ALLUREMENTS OR WHAT VANTAGES UPON THE FOREHEAD OF THE OTHERS SHOWED THAT THOU SHOULDST TURN THY FOOTSTEPS UNTO THEM
